---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [141]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0) # np.where: 条件に応じて値をセット。
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [142]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [38]:
 # Q1: balanced(データの配分が均等か)かどうかを%表示で取得します。
def answer_one():
    
    return len(spam_data[spam_data['target'] == 1]) / len(spam_data) * 100#Your answer here

In [39]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [149]:
 # Q2: 最も単語リストの中で文字数が多いものを取得します
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect_model = CountVectorizer().fit(X_train)
    
    # np.argmax: 最大値のindexを返す。 np.amax: 最大値を返す。ここではmax関数のkey引数にlenを渡してlongest wrdを取得する
    
    maxlen_word = max(vect_model.get_feature_names(), key = len)
    print(len(maxlen_word))
    return maxlen_word #Your answer here

In [150]:
answer_two()

32


'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [42]:
# Q3: AUCスコアを求める(CountVectorizer & Naive Bayes)
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    # 3-1 Fit and transform the training data X_train using a Count Vectorizer with default parameters.
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # 3-2  fit a multinomial Naive Bayes classifier model with smoothing alpha=0.1. 
    model = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    
    # 3-3  Find the area under the curve (AUC) score using the transformed test data
    X_test_vectorized = vect.transform(X_test) # ⇦ テスト時にはfitしない。transformのみ。
    predictions = model.predict(X_test_vectorized)

    
    return roc_auc_score(y_test, predictions)#Your answer here

In [43]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [44]:
# Q4: TF−IDFを求める(大きいものから２０と小さいものから20)
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    # 4-1 Fit and transform the training data X_train using a Tfidf Vectorizer with default parameters.
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    #print(X_train_vectorized)
    """
     出力:
      (0, 7305)	0.198150664334
      (0, 7085)	0.352009658683
      (0, 6421)	0.390987576266
      (0, 4624)	0.471125306271
                  :                   :
      (4178, 3336)	0.179845226367
      (4178, 3199)	0.275090466584
      (4178, 2065)	0.347443339363
      (4178, 931)	0.241310280193
      (4178, 790)	0.225106503401
    """    
    
    # 4-2 単語リスト取得
    feature_names = np.array(vect.get_feature_names())
#     print(len(feature_names)) ⇨ 7354
    
    # 4-3 What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?
    # maxを求めてsortすればminも求まる。(逆も然り)
    """
    次のロジックの説明。
    This max method is applied on a sparse matrix,
    so you'd have to look up the SciPy documentation to understand exactly how it works.
    
    But essentially, that 0 is passed to the method as the axis parameter.
    The axis 0 represent rows... so that would give you the max values in the sparse matrix for each row of said matrix.
    """
#     print(X_train_vectorized.max(axis=0) .shape, X_train_vectorized.max(axis=1) .shape) # 確認用　⇨ (1, 7354) (4179, 1)
    tfidf = X_train_vectorized.max(axis=0) # ⇦ <1x7354 sparse matrix of type '<class 'numpy.float64'>' with 0 stored elements in COOrdinate format>
    print(tfidf.shape)
    tfidf_a = tfidf.toarray().squeeze() # 階層が深いので[-1]以外使わない。(最後の並びのみ取る)
    sorted_index = tfidf_a.argsort() # 並び替えをする。maxの場合: array([6305, 3196,  689, ..., 3293, 3180,  274]) (minの場合: array([ 0, 4908, 4907, ..., 2444, 2442, 7353]) )
    print(sorted_index.shape)
    
#     print(len(sorted_index)) ⇨ 7354
#     print(feature_names[sorted_index]) ⇨  ['sympathetic' 'healer' 'aaniye' ..., 'home' 'havent' '146tf150p']
    
    smallest_tfidf_values = feature_names[sorted_index][:20]      # tfidfの最も大きいword20ずつを取得する
    largest_tfidf_values = feature_names[sorted_index][-20:]     # tfidfの最も小さいword20ずつを取得する
    
    # 低いtf-idfを示す用語はほぼどのような文書にも出てくる用語か、長い文章の中にしか登場しない用語(spamとの相関は関係無い)
    ans1 = pd.Series(tfidf_a[sorted_index][:20], index = smallest_tfidf_values)
    # 高いtf-idfを示す用語は特定の文書で頻出する用語(spamとの相関は関係無い)
    ans2 = pd.Series(tfidf_a[sorted_index][-20:], index = largest_tfidf_values)
    
    return ans1, ans2#Your answer here

In [45]:
answer_four()

(1, 7354)
(7354,)


(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, blank        0.932702
 tick         0.980166
 645          1.000000
 done         1.000000
 too          1.000000
 anytime      1.000000
 beerage      1.000000
 where        1.000000
 ok           1.000000
 thank        1.000000
 yup          1.000000
 nite         1.000000
 lei          1.000000
 anything     1.000000
 er           1.000000
 thanx        1.000000
 okie         1.000000
 home         1.000000
 havent       1.000000
 146tf150p    1.000000
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [46]:
# Q5: AUCスコアを求める(TfidfVectorizer & Naive Bayes)(Q3とあまり変わらない)
def answer_five():
    # Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 3.
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    X_test_vectorized = vect.transform(X_test)
    predictions = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, predictions)#Your answer here

In [47]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [48]:
# Q6: 文書の文字数の平均を求める
def answer_six():
#     spam_data['target'] = np.where(spam_data['target']=='spam',1,0) # np.where: 条件に応じて値をセット。
    
    spam_data['length'] = spam_data['text'].apply(len)

    print(spam_data.head(10))
    print(list(spam_data.iloc[1]['text'].split())) # word数は6つのみだが、文字数29は合ってる。
    print(len(list(spam_data.iloc[1]['text'])))

    df_spam_len = spam_data[spam_data['target'] == 1]['length'].mean()
    df_no_spam_len = spam_data[spam_data['target'] == 0]['length'].mean()
    
    return df_no_spam_len, df_spam_len#Your answer here

In [49]:
answer_six()

                                                text  target  length
0  Go until jurong point, crazy.. Available only ...       0     111
1                      Ok lar... Joking wif u oni...       0      29
2  Free entry in 2 a wkly comp to win FA Cup fina...       1     155
3  U dun say so early hor... U c already then say...       0      49
4  Nah I don't think he goes to usf, he lives aro...       0      61
5  FreeMsg Hey there darling it's been 3 week's n...       1     148
6  Even my brother is not like to speak with me. ...       0      77
7  As per your request 'Melle Melle (Oru Minnamin...       0     160
8  WINNER!! As a valued network customer you have...       1     158
9  Had your mobile 11 months or more? U R entitle...       1     154
['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']
29


(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [134]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [67]:
# Q7: AUCスコアを求める(TfidfVectorizer & SVM)
from sklearn.svm import SVC

def answer_seven():
    
    # 7-1 Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5.
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # 7-2 Using this document-term matrix and an additional feature, the length of document (number of characters)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(len))  # ⇦　 X_train.apply(len)は X_train.str.len()でもいいらしい
    
    # 7-3 fit a Support Vector Classification model with regularization C=10000
    model = SVC(C = 10000).fit(X_train_vectorized, y_train) # C: regularization(小さいほどよりregularizeする)
    
    # 7-4 compute the area under the curve (AUC) score using the transformed test data
    X_test_vectorized = vect.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(len))
    predictions = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, predictions)#Your answer here

In [68]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [137]:
# Q8: 文書中の数字の出現回数平均を求める
def answer_eight():
    
    print(list(spam_data.iloc[2]['text'].split()))
    print(spam_data[:3]['text'].str.findall('(\d+)'))
    print(spam_data[:3]['text'].str.findall('(\d)').str.len())
    
    spam_data['digit_len'] = spam_data['text'].str.findall('(\d)').str.len() # 数字の出現回数なので+が不要

    df_spam_len = spam_data[spam_data['target'] == 1]['digit_len'].mean()
    df_no_spam_len = spam_data[spam_data['target'] == 0]['digit_len'].mean()
    
    return df_no_spam_len, df_spam_len #Your answer here

In [138]:
answer_eight()

['Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question(std', 'txt', "rate)T&C's", 'apply', "08452810075over18's"]
0                                       []
1                                       []
2    [2, 21, 2005, 87121, 08452810075, 18]
Name: text, dtype: object
0     0
1     0
2    25
Name: text, dtype: int64


(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [139]:
# Q9: AUCスコアを求める(TfidfVectorizer & LogisticRegression)
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    # 9-1 Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5
    # and using word n-grams from n=1 to n=3 (unigrams, bigrams, and trigrams).
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # 9-2 Using this document-term matrix and an additional feature, the length of document (number of characters)
    # and number of digits per document
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(len))  # ⇦　 X_train.apply(len)は X_train.str.len()でもいいらしい
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.findall('(\d)').str.len())
    
    # 9-3 fit a Logistic Regression model with regularization C=100
    model = LogisticRegression(C = 100).fit(X_train_vectorized, y_train) # C: regularization(小さいほどよりregularizeする)
    
    # 9-4 compute the area under the curve (AUC) score using the transformed test data
    X_test_vectorized = vect.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(len))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.findall('(\d)').str.len())

    predictions = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, predictions) #Your answer here

In [140]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [147]:
# Q10: 文書中の文字以外の出現回数平均を求める
def answer_ten():
    
    # 10-1 What is the average number of non-word characters per document for not spam and spam documents?
    print(list(spam_data[:3]['text'].str.split()))
    print(spam_data[:3]['text'].str.findall('(\W+)'))
    print(spam_data[:3]['text'].str.findall('(\W)').str.len())
    
    spam_data['nonword_len'] = spam_data['text'].str.findall('(\W)').str.len() # 数字の出現回数なので+が不要

    df_spam_len = spam_data[spam_data['target'] == 1]['nonword_len'].mean()
    df_no_spam_len = spam_data[spam_data['target'] == 0]['nonword_len'].mean()
    
    return df_no_spam_len, df_spam_len#Your answer here

In [148]:
answer_ten()

[['Go', 'until', 'jurong', 'point,', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...'], ['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...'], ['Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question(std', 'txt', "rate)T&C's", 'apply', "08452810075over18's"]]
0    [ ,  ,  , , , .. ,  ,  ,  ,  ,  ,  ,  ,  ,  , ...
1                              [ , ... ,  ,  ,  , ...]
2    [ ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  , ...
Name: text, dtype: object
0    28
1    11
2    33
Name: text, dtype: int64


(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [171]:
def answer_eleven():
    
    # 11-1 Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than 5
    # and using word n-grams from n=2 to n=5 (unigrams, bigrams, and trigrams). To tell Count Vectorizer to use character n-grams
    # pass in analyzer='char_wb' which creates character n-grams only from text inside word boundaries.
    vect = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # 11-2 Using this document-term matrix and an additional feature, the length of document (number of characters)
    # and number of digits per document and number of non-word characters
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(len))  # ⇦　 X_train.apply(len)は X_train.str.len()でもいいらしい
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.findall('(\d)').str.len())
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.findall('(\W)').str.len())
    
    # 11-3 fit a Logistic Regression model with regularization C=100
    model = LogisticRegression(C = 100).fit(X_train_vectorized, y_train) # C: regularization(小さいほどよりregularizeする)
    
    # 11-4 compute the area under the curve (AUC) score using the transformed test data
    X_test_vectorized = vect.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(len))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.findall('(\d)').str.len())
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.findall('(\W)').str.len())

    predictions = model.predict(X_test_vectorized)
    auc = roc_auc_score(y_test, predictions)
    
    # 11-5 スパムと高い相関値を示す用語、低い相関値を示す用語を抽出する。
    sorted_coef_index = model.coef_[0].argsort() # 並び替えをする
    feature_names = np.array(vect.get_feature_names())
    
    # 11-6 11-4にてfeatureを３つ追加しているので対応する ['length_of_doc', 'digit_count', 'non_word_char_count']
    feature_names = np.append(feature_names, ['length_of_doc', 'digit_count', 'non_word_char_count'])
    print(len(feature_names), len(sorted_coef_index))
    print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index][:10]))
    print('Largest Coefs:\n{}\n'.format(feature_names[sorted_coef_index][-10:]))
    
    # 11-7 スパムと低い相関値を示す用語、高い相関値を示す用語をそれぞれ低い順と高い順に10個並べて返す
    """
    [::n] creates a list containing every n-th element of the original list, starting with the zeroth element.
    [::-n] does the same backwards, starting with the last element.
    tolist()を使うことで純粋なlistに変換する。
    """
    return auc, feature_names[sorted_coef_index][:10].tolist(), feature_names[sorted_coef_index][:-11:-1].tolist()#Your answer here

In [172]:
answer_eleven()

16317 16317
Smallest Coefs:
['. ' '..' '? ' ' i' ' y' ' go' ':)' ' h' 'go' ' m']

Largest Coefs:
['ar' 'ww' ' x' 'mob' ' ch' 'xt' 'co' 'ia' 'ne' 'digit_count']



(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])